[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
plt.rcParams['figure.dpi'] = 125

# Read the json file
with open("cases_deaths_vaccination.json", "rt") as INFILE:
    jsondata=json.load(INFILE)


def wrangle_data(rawdata):
    # Get the data and sort the date 
    datalist = rawdata['data']
    dates = [entry['date'] for entry in datalist]
    dates.sort()
    # Find the start date and end date after sorting them
    startdate = pd.to_datetime(dates[0], format="%Y-%m-%d")
    enddate = pd.to_datetime(dates[-1], format="%Y-%m-%d")
    # Now we are going to define the data frame
    # Set the dates between start and end date as the index then set some columns
    index = pd.date_range(startdate, enddate, freq='D')
    cases_deaths_vaccinationdf = pd.DataFrame(index=index, columns=['daily_cases', 
                                                                    'daily_deaths', 
                                                                    'total_cases', 
                                                                    'total_deaths', 
                                                                    'total_1st_dose', 
                                                                    'total_2nd_dose', 
                                                                    'total_booster_or_3rd_dose'])
    
    for entry in datalist: # Each entry is a dictionary with daily_cases, daily_deaths, total_cases, total_deaths, total_1st_dose, total_2nd_dose, total_booster_or_3rd_dose
        date = pd.to_datetime(entry['date'], format="%Y-%m-%d")
        for column in ['daily_cases', 
                       'daily_deaths', 
                       'total_cases', 
                       'total_deaths', 
                       'total_1st_dose', 
                       'total_2nd_dose', 
                       'total_booster_or_3rd_dose']:
            if pd.isna(cases_deaths_vaccinationdf.loc[date, column]): # replace None with 0 
                # Access a specific location in the data frame 
                # Put index and column in a single set of [ ]
                value = entry[column]
                cases_deaths_vaccinationdf.loc[date, column] = value 
    
    cases_deaths_vaccinationdf.fillna(0, inplace=True) # null replaced by 0
    # Calculate the total amount in 'month'
    # make sure that all the data look better (in a line shape) in the graph
    # especially for 'daily cases' and 'daily death' 
    cases_deaths_vaccinationdf = cases_deaths_vaccinationdf.resample('M').sum() 
    return cases_deaths_vaccinationdf

cases_deaths_vaccinationdf = wrangle_data(jsondata)


# Create a button, clicking it to call the function and refresh the data 
def access_api():
    return jsondata 

def api_button_callback(button):

    apidata = access_api()
    global cases_deaths_vaccinationdf
    cases_vaccinationdf=wrangle_data(apidata)
    refresh_graph(series.value, scale.value)
    apibutton.icon="refresh" 
    apibutton.disabled=False # It can be clicked again and again

apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success',
    tooltip="Press the button to refresh the data",
    icon='refresh'
)

apibutton.on_click(api_button_callback) # Register the callback function with the button
display(apibutton) # The way for Jupyter notebook to display the widget



def plot_new(new):
    # Create a plot for the data 0 to 6
    cases_deaths_vaccinationdf[new[0]].plot() 
    cases_deaths_vaccinationdf[new[1]].plot() 
    cases_deaths_vaccinationdf[new[2]].plot() 
    cases_deaths_vaccinationdf[new[3]].plot()
    cases_deaths_vaccinationdf[new[4]].plot()
    cases_deaths_vaccinationdf[new[5]].plot()
    cases_deaths_vaccinationdf[new[6]].plot()
    plt.show()

# Set a multiple choices widget for series
series=wdg.SelectMultiple(
    options=['daily_cases', 
             'daily_deaths', 
             'total_cases', 
             'total_deaths', 
             'total_1st_dose', 
             'total_2nd_dose', 
             'total_booster_or_3rd_dose'],
    value=[], #Here the default value is none
    rows=7,
    description='Option: ',
    disabled=False    
)


# Set a radio button widget for scale
scale=wdg.Dropdown(
    options=['linear', 'log'],
    value='linear',
    description='Scale:',
    disabled=False
)

# Set it to a VBox type
controls=wdg.HBox([series, scale])


# Create a function for refresh the data
def refresh_graph(gcols, gscale):

    series.value=[] # Back to the default (description for how to select the data)
    scale.value='linear'


# Callback function    
def cases_deaths_vaccination_graph(gcols, gscale):
    
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        cases_deaths_vaccinationdf[list(gcols)].plot(logy=logscale) # Change tuples to lists
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click or COMMAND-Click to select more than one category)")

# Keep calling cases_deaths_vaccination_graph(gcols=value_of_series, gscale=value_of_scale)
graph=wdg.interactive_output(cases_deaths_vaccination_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)


Button(button_style='success', description='Refresh data', icon='refresh', style=ButtonStyle(), tooltip='Press…

Output()

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).